In [1]:
# just spend an hour fixing a problem that didn't exist. All because Jan 1 2033 is a weekend...
# TO FIX:
# if im really getting into it: add a refi mode
# add percent principle and percent interest then represent graphically
# good example for graph output at url: https://www.calculator.net/amortization-calculator.html
# ***code breaks at term_year > 75 because of the turn of the century 

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, WEEKLY, MO
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
import calendar
from statistics import mean

# functions created for this project (should create links if I ever share this)
from validate_date import validate_date
from business_days import business_days


def loan_amortization(principal, interest_rate, start_date, term_years, periodtype):
    
    """ 
        Args:
        principal (int or float): The principal amount of the loan.
        interest_rate (int or float): The annual interest rate, expressed as a percent (e.g. 5% = 5).
        start_date (str or datetime): The start date of the loan in 'YYYY-MM-DD' format or as a datetime object.
        term_years (int or float): Loan/Borrowing term in years.
        periodtype (str): The type of period to use for the loan payments, which can be one of the following:
            'D' (daily)
            'bdays' (daily, only includes business days)
            'W' (weekly)
            'BW' (biweekly)
            'M-30' (months where there is 30 days per month and 360 days per year (30/360))
            'M-Actual' (months where months' length are accurate, and there are 360 days per year (Actual/360))
            'Q' (quarterly)
            'S' (semi-annual)
            'Y' (Annual)

    Returns:
        pandas.DataFrame: A DataFrame containing the amortization schedule for the loan, with columns for the payment number, payment date, payment amount, interest paid, principal paid, and balance.
    """

    if validate_date(start_date) == False:
        raise TypeError("start_date must be a string in 'YYYY-MM-DD' format or a datetime object")
    
    # if the date is in the string format, convert it
    if not isinstance(start_date, datetime):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    
    # input type checking for principal, interest_rate, and term_years
    if not isinstance(principal, (int, float)):
        raise TypeError("Principal amount should be numeric (int or float)")
    if not isinstance(interest_rate, (int, float)):
        raise TypeError("Interest rate should be numeric and in % (int or float)")
    if not isinstance(term_years, (int, float)):
        raise TypeError("term_years should be numeric (int or float)")
    
    # shift the day forward one when using Daily to assume no payment is made today
    if periodtype == "D":
        start_date = start_date + timedelta(days=1)
    
    # create end date of term using term_years
    end_date = start_date + timedelta(days=int(365.2422 * term_years))
    
    # create a list of business days for the bday index
    bdays_dates = business_days(start_date, end_date)
    
    # create a list of weeks for weekly and biweekly index
    mondays = list(rrule(WEEKLY, byweekday=MO, dtstart=start_date, until=end_date))
    
    # add one month so that payments are at start of next month
    if periodtype == 'M-30' or periodtype == 'M-Actual':
        if start_date.month == 12:
            # handle special case where the month is December
            new_month = 1
            new_year = start_date.year + 1
        else:
            new_month = start_date.month + 1
            new_year = start_date.year
        start_date_list = start_date.replace(year=new_year, month=new_month)
        month_dates = [start_date_list]
        current_date = start_date_list
        while current_date < end_date:
            current_date += relativedelta(months=1)
            month_dates.append(current_date)
    else:
        start_date_list = start_date
        month_dates = [start_date_list]
        current_date = start_date_list
        while current_date < end_date:
            current_date += relativedelta(months=1)
            month_dates.append(current_date)

    # create list of days in the month of each date
    days_in_month = [calendar.monthrange(date.year, date.month)[1] for date in month_dates]
    
    # period-type definition
    if periodtype == 'D':
        periods = int((end_date - start_date).days) + 1
        days_between_payments = 1
        adjusted_rate = interest_rate / 36524.22
    elif periodtype == 'bdays':
        periods = len(bdays_dates)
        adjusted_rate = interest_rate / 26100
    elif periodtype == 'W':
        periods = int(len(mondays))
        adjusted_rate = interest_rate / 5214
    elif periodtype == 'BW':
        periods = int(len(mondays[::2]))
        adjusted_rate = interest_rate / 2607
    elif periodtype == 'M-30':
        if start_date.month < end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1)
        elif start_date.month > end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 1)
        else:
            periods = int((end_date.year - start_date.year) * 12)
        adjusted_rate = interest_rate / 1200
    elif periodtype == 'M-Actual':
        if start_date.month < end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1)
        elif start_date.month > end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 1)
        else:
            periods = int((end_date.year - start_date.year) * 12)
        monthly_rate = [interest_rate / 36000 * days_in_month[i] for i in range(len(month_dates))]
        adjusted_rate = mean(monthly_rate)
    elif periodtype == 'Y':
        periods = int(end_date.year - start_date.year) + 1
        adjusted_rate = interest_rate / 100
    elif periodtype == 'S':
        periods = int(len(month_dates[1::6]))
        adjusted_rate = interest_rate / 200
    elif periodtype == 'Q':
        periods = int(len(month_dates[1::3]))
        adjusted_rate = interest_rate / 400
    else:
        raise TypeError("periodtype should be one of the following: 'D', 'W', 'BW', 'bdays', 'M-30', 'M-Actual', 'Q', 'S', 'Y'")

    monthly_payment = principal * adjusted_rate / (1 - (1 + adjusted_rate) ** (-periods))

    # create a list of dates for each payment
    if periodtype == 'M-Actual' or periodtype == 'M-30':
        payment_dates = month_dates
    elif periodtype == 'Y':
        payment_dates = [(start_date + relativedelta(years=1 * i)).year for i in range(periods)]
    elif periodtype == 'bdays':
        payment_dates = bdays_dates
    elif periodtype == 'W':
        payment_dates = mondays
    elif periodtype == 'BW':
        payment_dates = mondays[::2]
    elif periodtype == 'S':
        payment_dates = month_dates[6::6]
    elif periodtype == 'Q':
        payment_dates = month_dates[3::3]
    else:
        payment_dates = [start_date + relativedelta(days=(days_between_payments * i)) for i in range(periods)]

    # lists for the payment number, payment amount, interest, principal, and balance
    payment_number = list(range(1, periods + 1))
    payment_amount = [monthly_payment] * periods
    interest = []
    principal_paid = []
    beg_balance = [principal]
    end_balance = []

    # interest, principal, and balance for each payment period (exlcuding M-actual)
    if not periodtype == "M-Actual":
        for i in range(periods):
            interest.append(beg_balance[i] * adjusted_rate)
            principal_paid.append(monthly_payment - interest[i])
            beg_balance.append(beg_balance[i] - principal_paid[i])
            end_balance.append(beg_balance[i] - principal_paid[i])
    elif periodtype == "M-Actual":
        for i in range(periods):
            interest.append(beg_balance[i] * monthly_rate[i])
            principal_paid.append(monthly_payment - interest[i])
            beg_balance.append(beg_balance[i] - principal_paid[i])
            end_balance.append(beg_balance[i] - principal_paid[i])
        principal_paid[-1] = beg_balance[-2]
        payment_amount[-1] = principal_paid[-1] + interest[-1]
        end_balance[-1] = 0

    # make the amortization-schedule dataframe
    data = {
        'Payment Number': payment_number,
        'Payment Date': payment_dates,
        'Beginning Balance': beg_balance[:-1],
        'Payment Amount': payment_amount,
        'Interest Paid': interest,
        'Principal Paid': principal_paid,
        'Ending Balance': end_balance
    }
    df = pd.DataFrame(data)

    # set the index to the payment date
    df.set_index('Payment Date', inplace=True)

    # apply formating for dollar signs and two decimals
    df.index.name = 'Payment Date'
    df['Payment Amount'] = df['Payment Amount'].apply(lambda x: '${:,.2f}'.format(x))
    df['Interest Paid'] = df['Interest Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Principal Paid'] = df['Principal Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Beginning Balance'] = df['Beginning Balance'].apply(lambda x: '${:,.2f}'.format(x))
    df['Ending Balance'] = df['Ending Balance'].apply(lambda x: '${:,.2f}'.format(x))

    return df

In [2]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'D')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-01-02,1,"$10,000.00",$3.48,$1.37,$2.11,"$9,997.89"
2023-01-03,2,"$9,997.89",$3.48,$1.37,$2.11,"$9,995.78"
2023-01-04,3,"$9,995.78",$3.48,$1.37,$2.11,"$9,993.67"
2023-01-05,4,"$9,993.67",$3.48,$1.37,$2.11,"$9,991.56"
2023-01-06,5,"$9,991.56",$3.48,$1.37,$2.11,"$9,989.45"
...,...,...,...,...,...,...
2032-12-28,3649,$17.39,$3.48,$0.00,$3.48,$13.91
2032-12-29,3650,$13.91,$3.48,$0.00,$3.48,$10.43
2032-12-30,3651,$10.43,$3.48,$0.00,$3.48,$6.96


In [3]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'bdays')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-01-02,1,"$10,000.00",$4.87,$1.92,$2.95,"$9,997.05"
2023-01-03,2,"$9,997.05",$4.87,$1.92,$2.95,"$9,994.09"
2023-01-04,3,"$9,994.09",$4.87,$1.91,$2.95,"$9,991.14"
2023-01-05,4,"$9,991.14",$4.87,$1.91,$2.96,"$9,988.18"
2023-01-06,5,"$9,988.18",$4.87,$1.91,$2.96,"$9,985.23"
...,...,...,...,...,...,...
2032-12-27,2606,$24.33,$4.87,$0.00,$4.86,$19.47
2032-12-28,2607,$19.47,$4.87,$0.00,$4.87,$14.60
2032-12-29,2608,$14.60,$4.87,$0.00,$4.87,$9.74


In [4]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'W')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-01-02,1,"$10,000.00",$24.36,$9.59,$14.77,"$9,985.23"
2023-01-09,2,"$9,985.23",$24.36,$9.58,$14.78,"$9,970.45"
2023-01-16,3,"$9,970.45",$24.36,$9.56,$14.80,"$9,955.65"
2023-01-23,4,"$9,955.65",$24.36,$9.55,$14.81,"$9,940.84"
2023-01-30,5,"$9,940.84",$24.36,$9.53,$14.83,"$9,926.01"
...,...,...,...,...,...,...
2032-11-29,518,$121.45,$24.36,$0.12,$24.24,$97.20
2032-12-06,519,$97.20,$24.36,$0.09,$24.27,$72.94
2032-12-13,520,$72.94,$24.36,$0.07,$24.29,$48.65


In [5]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'BW')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-01-02,1,"$10,000.00",$48.74,$19.18,$29.56,"$9,970.44"
2023-01-16,2,"$9,970.44",$48.74,$19.12,$29.61,"$9,940.83"
2023-01-30,3,"$9,940.83",$48.74,$19.07,$29.67,"$9,911.16"
2023-02-13,4,"$9,911.16",$48.74,$19.01,$29.73,"$9,881.43"
2023-02-27,5,"$9,881.43",$48.74,$18.95,$29.78,"$9,851.65"
...,...,...,...,...,...,...
2032-10-25,257,$242.29,$48.74,$0.46,$48.27,$194.01
2032-11-08,258,$194.01,$48.74,$0.37,$48.36,$145.65
2032-11-22,259,$145.65,$48.74,$0.28,$48.46,$97.19


In [6]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'Q')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-04-01,1,"$10,000.00",$319.21,$125.00,$194.21,"$9,805.79"
2023-07-01,2,"$9,805.79",$319.21,$122.57,$196.64,"$9,609.14"
2023-10-01,3,"$9,609.14",$319.21,$120.11,$199.10,"$9,410.04"
2024-01-01,4,"$9,410.04",$319.21,$117.63,$201.59,"$9,208.46"
2024-04-01,5,"$9,208.46",$319.21,$115.11,$204.11,"$9,004.35"
2024-07-01,6,"$9,004.35",$319.21,$112.55,$206.66,"$8,797.69"
2024-10-01,7,"$8,797.69",$319.21,$109.97,$209.24,"$8,588.44"
2025-01-01,8,"$8,588.44",$319.21,$107.36,$211.86,"$8,376.59"
2025-04-01,9,"$8,376.59",$319.21,$104.71,$214.51,"$8,162.08"


In [7]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'S')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-07-01,1,"$10,000.00",$641.47,$250.00,$391.47,"$9,608.53"
2024-01-01,2,"$9,608.53",$641.47,$240.21,$401.26,"$9,207.27"
2024-07-01,3,"$9,207.27",$641.47,$230.18,$411.29,"$8,795.98"
2025-01-01,4,"$8,795.98",$641.47,$219.90,$421.57,"$8,374.41"
2025-07-01,5,"$8,374.41",$641.47,$209.36,$432.11,"$7,942.30"
2026-01-01,6,"$7,942.30",$641.47,$198.56,$442.91,"$7,499.38"
2026-07-01,7,"$7,499.38",$641.47,$187.48,$453.99,"$7,045.40"
2027-01-01,8,"$7,045.40",$641.47,$176.13,$465.34,"$6,580.06"
2027-07-01,9,"$6,580.06",$641.47,$164.50,$476.97,"$6,103.09"


In [8]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'Y')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023,1,"$10,000.00","$1,295.05",$500.00,$795.05,"$9,204.95"
2024,2,"$9,204.95","$1,295.05",$460.25,$834.80,"$8,370.16"
2025,3,"$8,370.16","$1,295.05",$418.51,$876.54,"$7,493.62"
2026,4,"$7,493.62","$1,295.05",$374.68,$920.36,"$6,573.25"
2027,5,"$6,573.25","$1,295.05",$328.66,$966.38,"$5,606.87"
2028,6,"$5,606.87","$1,295.05",$280.34,"$1,014.70","$4,592.17"
2029,7,"$4,592.17","$1,295.05",$229.61,"$1,065.44","$3,526.73"
2030,8,"$3,526.73","$1,295.05",$176.34,"$1,118.71","$2,408.02"
2031,9,"$2,408.02","$1,295.05",$120.40,"$1,174.64","$1,233.38"


In [9]:
# Test: 5-year loan, starting on January 1, 2023 and ending on December 31, 2027
schedule = loan_amortization(10000, 5, "2023-1-1", 30, 'M-30')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-02-01,1,"$10,000.00",$53.68,$41.67,$12.02,"$9,987.98"
2023-03-01,2,"$9,987.98",$53.68,$41.62,$12.07,"$9,975.92"
2023-04-01,3,"$9,975.92",$53.68,$41.57,$12.12,"$9,963.80"
2023-05-01,4,"$9,963.80",$53.68,$41.52,$12.17,"$9,951.64"
2023-06-01,5,"$9,951.64",$53.68,$41.47,$12.22,"$9,939.42"
...,...,...,...,...,...,...
2052-09-01,356,$265.09,$53.68,$1.10,$52.58,$212.51
2052-10-01,357,$212.51,$53.68,$0.89,$52.80,$159.71
2052-11-01,358,$159.71,$53.68,$0.67,$53.02,$106.70


In [10]:
# Test: 5-year loan, starting on January 1, 2023 and ending on December 31, 2027
schedule = loan_amortization(10000, 5, "2023-1-1", 30, 'M-Actual')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Payment Date,,,,,,
2023-02-01,1,"$10,000.00",$54.13,$38.89,$15.24,"$9,984.76"
2023-03-01,2,"$9,984.76",$54.13,$42.99,$11.14,"$9,973.62"
2023-04-01,3,"$9,973.62",$54.13,$41.56,$12.57,"$9,961.05"
2023-05-01,4,"$9,961.05",$54.13,$42.89,$11.24,"$9,949.80"
2023-06-01,5,"$9,949.80",$54.13,$41.46,$12.67,"$9,937.13"
...,...,...,...,...,...,...
2052-09-01,356,$258.94,$54.13,$1.08,$53.05,$205.89
2052-10-01,357,$205.89,$54.13,$0.89,$53.24,$152.65
2052-11-01,358,$152.65,$54.13,$0.64,$53.49,$99.15
